# Quick introduction

gliderpy can be installed with conda

```
conda install --channel conda-forge gliderpy
```

or pip

```
pip install gliderpy
```

gliderpy aims to make querying and downloading glider data easier.
Here is how one would build a query using erddapy:

In [1]:
from erddapy import ERDDAP

e = ERDDAP(
    server="https://gliders.ioos.us/erddap",
    protocol="tabledap",
    response="csv",
)
e.dataset_id = "whoi_406-20160902T1700"

e.variables = [
    "depth",
    "latitude",
    "longitude",
    "salinity",
    "temperature",
    "profile_id",
    "time",
]

df = e.to_pandas(
    index_col="time (UTC)",
    parse_dates=True,
)
df.head()

,depth (m),latitude (degrees_north),longitude (degrees_east),salinity (1),temperature (Celsius),profile_id
time (UTC),,,,,,
2016-09-02 17:05:38+00:00,6.43,41.120018,-71.017118,32.079258,20.5407,1
2016-09-02 17:05:38+00:00,7.82,41.120018,-71.017118,32.095406,20.5071,1
2016-09-02 17:05:38+00:00,8.69,41.120018,-71.017118,32.111920,20.4904,1
2016-09-02 17:05:38+00:00,9.54,41.120018,-71.017118,32.126587,20.4897,1
2016-09-02 17:05:38+00:00,10.32,41.120018,-71.017118,32.140717,20.4909,1


And here is how to use gliderpy to obtain the same results but with fewer lines and a cleaner code:

In [2]:
from gliderpy.fetchers import GliderDataFetcher

glider_grab = GliderDataFetcher()

glider_grab.fetcher.dataset_id = "whoi_406-20160902T1700"
df = glider_grab.to_pandas()
df.head()

,latitude,longitude,pressure,profile_id,salinity,temperature,dataset_url
time (utc),,,,,,,
2016-09-02 17:05:38+00:00,41.120018,-71.017118,6.43,1,32.079258,20.5407,https://gliders.ioos.us/erddap/tabledap/whoi_4...
2016-09-02 17:05:38+00:00,41.120018,-71.017118,7.82,1,32.095406,20.5071,https://gliders.ioos.us/erddap/tabledap/whoi_4...
2016-09-02 17:05:38+00:00,41.120018,-71.017118,8.69,1,32.111920,20.4904,https://gliders.ioos.us/erddap/tabledap/whoi_4...
2016-09-02 17:05:38+00:00,41.120018,-71.017118,9.54,1,32.126587,20.4897,https://gliders.ioos.us/erddap/tabledap/whoi_4...
2016-09-02 17:05:38+00:00,41.120018,-71.017118,10.32,1,32.140717,20.4909,https://gliders.ioos.us/erddap/tabledap/whoi_4...


Much easier, right?
The variable names are standardized by gliderpy,
making it easier to fetch from different data sources and comparing the results.

The gliderpy library can subset the data on the server side by passing a geographic bounding box and time interval.

### Querying multiple datasets

The most common use is to search all datasets for data that falls within the certain space-time bounds.

In [3]:
glider_grab = GliderDataFetcher()

df = glider_grab.query(10, 40, -90, 8, "2010-01-01", "2013-06-02")
df

,Title,Institution,Dataset ID,info_url
0,ru23-20130305T2004,Rutgers University,ru23-20130305T2004,https://gliders.ioos.us/erddap/info/ru23-20130...
1,ru23-20121025T1944,Rutgers University,ru23-20121025T1944,https://gliders.ioos.us/erddap/info/ru23-20121...


In [4]:
datasets = glider_grab.to_pandas()
datasets.keys()

dict_keys(['ru23-20130305T2004', 'ru23-20121025T1944'])

In [5]:
datasets["ru23-20121025T1944"].head()

,latitude,longitude,pressure,profile_id,salinity (1e-3),temperature,dataset_url
time (utc),,,,,,,
2012-10-27 18:20:25+00:00,39.999821,-73.510566,1.67,289,31.973030,16.9787,https://gliders.ioos.us/erddap/tabledap/ru23-2...
2012-10-27 18:20:25+00:00,39.999821,-73.510566,1.68,289,31.973806,16.9775,https://gliders.ioos.us/erddap/tabledap/ru23-2...
2012-10-27 18:20:25+00:00,39.999821,-73.510566,NaN,289,NaN,NaN,https://gliders.ioos.us/erddap/tabledap/ru23-2...
2012-10-27 18:20:25+00:00,39.999821,-73.510566,1.82,289,31.973087,16.9778,https://gliders.ioos.us/erddap/tabledap/ru23-2...
2012-10-27 18:20:25+00:00,39.999821,-73.510566,NaN,289,NaN,NaN,https://gliders.ioos.us/erddap/tabledap/ru23-2...


### Dataset search

One can query all dataset_ids available in the server.

In [6]:
from gliderpy.fetchers import DatasetList

datasets = DatasetList()
ds_ids = datasets.get_ids()

print(f"found {len(ds_ids)} glider datasets on the server {datasets.e.server}.")

found 1770 glider datasets on the server https://gliders.ioos.us/erddap.
